In [1]:
%load_ext autoreload
%autoreload
import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>
    div.output_area{
        max-height:10000px;
        overflow:scroll;
    }
</style>"""))

import shapefile
import pygeohash as geohash
from timeUtils import clock, elapsed
from shapely.geometry.polygon import Polygon
from shapely.geometry import Point
from random import uniform
from geoUtils import *
from censusUtils import getCensusPoints, getCensusData, getCensusName
basedir="/Users/tgadf/Downloads/Census"

Notebook Last Run Initiated: 2018-10-18 09:43:36.784593


In [19]:
globalVar = None
prec = 6
stdirs=[]
if True:
    #stdirs.append("State_2010Census_DP1")
    stdirs.append("County_2010Census_DP1")
    stdirs.append("CBSA_2010Census_DP1")
    stdirs.append("CSA_2010Census_DP1")
    stdirs.append("Place_2010Census_DP1")
    stdirs.append("MetDiv_2010Census_DP1")
    stdirs.append("CouSub_2010Census_DP1")
    #stdirs.append("ZCTA_2010Census_DP1")
    #stdirs.append("Tract_2010Census_DP1")

# Geohashs By Geoid

In [20]:
from os.path import join
from timeUtils import clock, elapsed
debug = False

todo = {"Records": True, "Geos": False}

for stdir in stdirs:
    
    geoShapeMap = {}
    shapeData = {}
    
    sf = shapefile.Reader(join(basedir, stdir, stdir))
    
    ipop,ihouse,ihouseocc = getCensusPoints(sf, show=False)

    startall,cmtall = clock("Geohashing all {0} shapes".format(stdir))
    ngeos = 0
    irec  = 0
    areas = []
    for shapeRec in sf.iterShapeRecords():
        irec += 1

        ####################################################################
        ## Record
        ####################################################################
        record = shapeRec.record
        geoid  = record[0]
        state  = geoid[:2]
        name   = record[1]
        memi   = record[2]
        area   = record[-1]
        pops   = record[ipop:ipop+19]
        house  = record[ihouse:ihouse+15]
        occpy  = record[ihouseocc:ihouseocc+9]

        name,memi = getCensusName(stdir, name, memi)
        recdata   = getCensusData(pops, house, occpy)

        if geoid in ['0201601615']:
            continue
        
        if todo["Records"] is True:
            shapeData[geoid]   = {"Name": name}
            shapeData[geoid].update(recdata)
            if irec % 100 == 0:
                print(irec,'\t',geoid,'\t',name,'\t',area)
            
            

        if todo["Geos"] is True:
            ####################################################################
            ## Geohashs (Shape)
            ####################################################################
            shape = shapeRec.shape
            maxPrec = 7
            if area > 10:
                maxPrec = 4
            elif area > 2:
                maxPrec = 5
            elif area > 0.5:
                maxPrec = 6
            areas.append(area)
            
            if debug:
                start, cmt = clock("Geohashing Shape with name {0}".format(name))
            
            geos  = getGeos(shape, prec=6, returnKeys=True)
            #geos  = getShapeAndGeos(shape, maxPrec=maxPrec, debug=False)
            if debug:
                elapsed(start, cmt)
            geoShapeMap[geoid] = geos
            print(irec,'\t',geoid,'\t',name,'\t',area,'\t--->',len(geos))
            if len(geos) <= 2:
                print("---------------------> {0} has less than 2 geoids <---------------------".format(geoid))
            ngeos += len(geos)

        
        if irec % 250 == 0 and False:
            print("Found {0} records with {1} geos of type {2}".format(len(shapeData), ngeos, stdir))
            saveGeoData(shapeData, geoShapeMap, 0, ngeos, "Census/{0}-{1}-tmp".format(stdir, prec))
        

    print("Found {0} records with {1} geos of type {2}".format(len(shapeData), ngeos, stdir))
    saveGeoData(shapeData, geoShapeMap, 0, ngeos, "Census/{0}-{1}".format(stdir, prec))
                    

    elapsed(startall,cmtall)


Current Time is Thu Oct 18, 2018 10:05:57 for Geohashing all County_2010Census_DP1 shapes
100 	 06059 	 Orange County 	 0.238729607709
200 	 48417 	 Shackelford County 	 0.228122474125
300 	 42047 	 Elk County 	 0.232194891506
400 	 38087 	 Slope County 	 0.369692725622
500 	 72069 	 Humacao Municipio 	 0.0157874645105
600 	 36009 	 Cattaraugus County 	 0.373648647678
700 	 38021 	 Dickey County 	 0.344237847745
800 	 49009 	 Daggett County 	 0.199425244024
900 	 48029 	 Bexar County 	 0.302531987356
1000 	 20023 	 Cheyenne County 	 0.27802283365
1100 	 13229 	 Pierce County 	 0.0842678066925
1200 	 26163 	 Wayne County 	 0.190162617554
1300 	 54035 	 Jackson County 	 0.126727004172
1400 	 13175 	 Laurens County 	 0.20332702923
1500 	 51169 	 Scott County 	 0.140707668873
1600 	 20183 	 Smith County 	 0.244258295293
1700 	 21101 	 Henderson County 	 0.123536903133
1800 	 39081 	 Jefferson County 	 0.112884060748
1900 	 51153 	 Prince William County 	 0.0934639692505
2000 	 16019 	 Bonn

9200 	 2032575 	 Holcomb city 	 0.000357883908499
9300 	 2009675 	 Bushong city 	 4.00590534994e-05
9400 	 2039450 	 Leon city 	 0.000198013164501
9500 	 2054575 	 Parkerville city 	 3.22511774995e-05
9600 	 2022775 	 Fall River city 	 5.73313534994e-05
9700 	 2034625 	 Iuka city 	 0.000158257512499
9800 	 2161788 	 Pleasureville city 	 0.000124902652
9900 	 2168556 	 Sandy Hook city 	 0.000261500318999
10000 	 2167458 	 Russell city 	 0.000809474881499
10100 	 2180706 	 Warsaw city 	 0.000186313388501
10200 	 2156694 	 North Corbin CDP 	 0.0004711868115
10300 	 2241400 	 Lake Providence town 	 0.000901287888498
10400 	 2241155 	 Lake Charles city 	 0.010870227275
10500 	 2214485 	 Chatham town 	 0.000285488678999
10600 	 2264100 	 Reeves village 	 0.0005890718465
10700 	 2228065 	 Galliano CDP 	 0.0026855562875
10800 	 2305350 	 Blaine CDP 	 0.0008286213485
10900 	 2324480 	 Falmouth CDP 	 0.000944961888
11000 	 2479675 	 University Park town 	 0.000134583357
11100 	 2444525 	 Klondik

25300 	 4850724 	 Nesbitt town 	 0.0004343535315
25400 	 4807852 	 Berryville town 	 0.000323934157499
25500 	 4805732 	 Bartlett city 	 0.000297370648
25600 	 4851336 	 New Summerfield city 	 0.001161478381
25700 	 4828128 	 Garceno CDP 	 5.52691819999e-05
25800 	 4834133 	 Hilltop CDP 	 1.15327095001e-05
25900 	 4844142 	 Los Lobos CDP 	 3.33271650002e-05
26000 	 4848324 	 Milam CDP 	 0.0098097791
26100 	 4853004 	 Oak Island CDP 	 0.0002898720055
26200 	 4867766 	 Sienna Plantation CDP 	 0.0033827425045
26300 	 4915940 	 Cornish town 	 0.0013708850075
26400 	 4911440 	 Cedar Hills city 	 0.000757277117501
26500 	 4939040 	 Jensen CDP 	 0.000887015163499
26600 	 5008950 	 Bristol CDP 	 0.0006277142
26700 	 5166896 	 Rich Creek town 	 0.0002235245685
26800 	 5117952 	 Coeburn town 	 0.000542780231499
26900 	 5108472 	 Bon Air CDP 	 0.002205850835
27000 	 5110856 	 Buckingham Courthouse CDP 	 0.000296010041
27100 	 5126736 	 Fairlawn CDP 	 0.000900907107999
27200 	 5148445 	 McMullin C

9400 	 2011952075 	 Odee township 	 0.0249610618845
9500 	 2012330975 	 Hayes township 	 0.0097475421105
9600 	 2019173350 	 Valverde township 	 0.009470610235
9700 	 2122793084 	 Sand Hill-Girkin CCD 	 0.015117142057
9800 	 2111592600 	 Oil Springs CCD 	 0.014059064022
9900 	 2102992008 	 Lebanon Junction CCD 	 0.017159808132
10000 	 2103193392 	 Sugar Grove CCD 	 0.022249510526
10100 	 2105993556 	 Utica-Stanley CCD 	 0.067610903096
10200 	 2208795746 	 District A 	 0.001870587593
10300 	 2211195065 	 District 6 	 0.0320877928925
10400 	 2201595368 	 District 9 	 0.000631457638
10500 	 2203194423 	 District 3 	 0.0138253682305
10600 	 2207795641 	 District 12 	 0.002064072523
10700 	 2301910810 	 Carroll plantation 	 0.013135581695
10800 	 2302985185 	 Whiting town 	 0.0153301130815
10900 	 2300972865 	 Southwest Harbor town 	 0.0066227296605
11000 	 2300749520 	 New Vineyard town 	 0.0106694834725
11100 	 2302129535 	 Greenville town 	 0.013746777865
11200 	 2300528240 	 Gorham town

24300 	 3914564304 	 Portsmouth city 	 0.002971729713
24400 	 3902779772 	 Vernon township 	 0.0075920054795
24500 	 3911315518 	 Clay township 	 0.0102744481675
24600 	 3909316910 	 Columbia township 	 0.0071631900515
24700 	 3906101980 	 Anderson township 	 0.008410161355
24800 	 3901364962 	 Pultney township 	 0.00711959263
24900 	 3915143022 	 Lexington township 	 0.0067795731615
25000 	 3911733600 	 Harmony township 	 0.0065842887555
25100 	 3914776778 	 Tiffin city 	 0.001915953592
25200 	 3904332578 	 Groton township 	 0.0071782148565
25300 	 3913779450 	 Van Buren township 	 0.010085928282
25400 	 3911937926 	 Jackson township 	 0.006684844935
25500 	 3909945178 	 Lowellville village 	 0.000400051438501
25600 	 3917169638 	 St. Joseph township 	 0.0098188537075
25700 	 3908328658 	 Fredericktown village 	 0.000571093626501
25800 	 3903526446 	 Fairview Park city 	 0.001305461239
25900 	 4002391430 	 Hugo CCD 	 0.0651662229865
26000 	 4007190325 	 Braman CCD 	 0.033729619952
261

# Combine Everything

In [25]:

for stdir in stdirs:
    
    try:
        recname = "{0}/{1}-{2}-data.p".format(basedir, stdir, prec)
        recdata = pickle.load(open(recname, "rb"))
        print("Found {0} records".format(len(recdata)))
    except:
        print("Could not open {0}".format(recname))
        continue
    try:
        geoname = "{0}/{1}-{2}-geos.p".format(basedir, stdir, prec)
        geodata = pickle.load(open(geoname, "rb"))
        print("Found {0} geo records".format(len(geodata)))
    except:
        print("Could not open {0}".format(geoname))
        continue
    ngeos = 0

    

    geomap   = {}
    records  = {}
    ngeos    = 0
    totalgeos = 0

    for geoid,geos in geodata.items():
        ngeos += len(geos)
        totalgeos += ngeos
        for geo in geos:
            if geomap.get(geo) is None:
                geomap[geo] = {}
                geomap[geo][stdir.replace("_2010Census_DP1", "")] = geoid

    for geoid,rec in recdata.items():
        records[geoid] = rec

    print("{0}\t{1}\t---> {2} <---".format(stdir,ngeos,len(geomap)))

        

    #for geo in geomap.keys():
    #    geomap[geo] = dict(geomap[geo])
        
    fname="geomap-{0}-{1}.p".format(prec, stdir.replace("_2010Census_DP1", ""))
    print("Writing {0}".format(fname))
    pickle.dump(geomap,  open(fname, "wb"))

    fname="georec-{0}-{1}.p".format(prec, stdir.replace("_2010Census_DP1", ""))
    print("Writing {0}".format(fname))
    pickle.dump(records,  open(fname, "wb"))

Found 3221 records
Could not open /Users/tgadf/Downloads/Census/County_2010Census_DP1-6-geos.p
Found 955 records
Found 955 geo records
CBSA_2010Census_DP1	1651995	---> 1529468 <---
Writing geomap-6-CBSA.p
Writing georec-6-CBSA.p
Found 128 records
Found 128 geo records
CSA_2010Census_DP1	399363	---> 389760 <---
Writing geomap-6-CSA.p
Writing georec-6-CSA.p
Found 29514 records
Found 29514 geo records
Place_2010Census_DP1	1558358	---> 1412116 <---
Writing geomap-6-Place.p
Writing georec-6-Place.p
Found 29 records
Found 29 geo records
MetDiv_2010Census_DP1	64511	---> 61746 <---
Writing geomap-6-MetDiv.p
Writing georec-6-MetDiv.p
Found 36641 records
Found 36641 geo records
CouSub_2010Census_DP1	21008559	---> 19416122 <---
Writing geomap-6-CouSub.p
Writing georec-6-CouSub.p


In [23]:
if True:
    from modelio import loadJoblib
    recs = loadJoblib("georec-6-CSA_2010Census_DP1.p")
    print(len(recs))
    for k,v in recs.items():
        print(k,v)
        break

    data = loadJoblib("geomap-6-CSA_2010Census_DP1.p")
    print(len(data))
    for k,v in data.items():
        if len(v) < 10:
            print('geo ->',k)
            print('geoid ->',v)
            print(recs.get(v))
            break

128
206 {'Name': 'Dallas-Fort Worth, TX CSA', 'Housing': [2437829, 0.69], 'Occupancy': [2670033, 0.91], 'Pop': [6731317, 0.3, 0.5, 0.19]}
389760
geo -> 9y5809
geoid -> 206
{'Name': 'Dallas-Fort Worth, TX CSA', 'Housing': [2437829, 0.69], 'Occupancy': [2670033, 0.91], 'Pop': [6731317, 0.3, 0.5, 0.19]}


# Plot Specific Shape

In [11]:
from os.path import join
stdir = stdirs[0]
shapename = join(basedir, stdir, stdir)
inputs = getShapeAndGeos(shapename, '30100', debug=False)

In [10]:
import folium
center  = None
geoPnts = None

sdata   = inputs["Shape"]
if sdata is None:
    print("Shape data is NONE!")
geos    = inputs["Geos"]
if geos is None:
    print("Boundary geos data is NONE!")
intgeos = inputs["IntGeos"]
if intgeos is None:
    print("Internal geos data is NONE!")

from geoClusteringPlots import drawClusters
dc = drawClusters()

if sdata is not None:
    print("Drawing shape with {0} points".format(sdata))
    from shapely.geometry.polygon import Polygon
    from shapely.geometry import Point
    poly      = Polygon(sdata)
    polyarea  = poly.area
    try:
        center    = poly.representative_point()
    except:
        center    = poly.centroid
    center    = (center.x, center.y)

if geos is not None:
    geoPnts = []
    print("Drawing {0}".format(len(geos)))
    for geo in geos:
        geoPnts.append(dc.getBB(geo, istuple=True))

if intgeos is not None:
    intGeoPnts = []
    print("Drawing {0}".format(len(intgeos)))
    for geo in intgeos:
        intGeoPnts.append(dc.getBB(geo, istuple=True))

extraGeos=[]
extraGeoPnts=[]
for geo in extraGeos:
    extraGeoPnts.append(dc.getBB(geo, istuple=True))
        
if center is not None:
    m = folium.Map(location=center, zoom_start=11)

    if sdata is not None:
        folium.PolyLine(sdata, color='blue', weight=1, opacity=1).add_to(m)
    if geoPnts is not None:
        folium.PolyLine(geoPnts, color='red', weight=2, opacity=1).add_to(m)
    if intGeoPnts is not None:
        folium.PolyLine(intGeoPnts, color='purple', weight=1, opacity=1).add_to(m)
    if extraGeoPnts is not None:
        folium.PolyLine(extraGeoPnts, color='green', weight=1.5, opacity=1).add_to(m)

    m.save("test.html")
m

Shape data is NONE!
Boundary geos data is NONE!
Internal geos data is NONE!


NameError: name 'm' is not defined

# Records By Geoid

In [ ]:
1.4/.35

In [ ]:
0.35/.044

In [ ]:
getGeoLngWidth(2)

In [17]:
%load_ext autoreload
%autoreload
import folium
from geocode import getGeoBBox, getGeoPrimBBox, getGeoDiagBBox, getGeoQuadBBox, getGeoOctoBBox, getGeoDeciBBox, getCenterFromGeo, getGeoIntPrimBox, getGeoIntDiagBox
from pygeohash import decode, getGeoChars


#########################################################################################################
geos = ['9vyw']
bbox = getGeoBBox(geos[0])
lat,lng = decode(geos[0])[:2]

print(lat,lng)
m = folium.Map(location=[lat,lng], zoom_start=9)
folium.PolyLine(bbox, color='darkred', weight=2, opacity=1).add_to(m)
for geo in ["{0}{1}".format(geos[0], x) for x in getGeoChars()]:
    bbox = getGeoBBox(geo)
    folium.PolyLine(bbox, color='red', weight=1, opacity=1).add_to(m)
#########################################################################################################



#########################################################################################################
sdata = inputs['Shape']
if sdata is not None:
    folium.PolyLine(sdata, color='blue', weight=1, opacity=1).add_to(m)
#########################################################################################################


for geo in geos:
    pnts = getGeoIntPrimBox(geo)
    for pnt in pnts:
        print(pnt)
        folium.Circle(pnt, color='darkblue', radius=1500, fill=False, weight=1, opacity=1).add_to(m)
      
    pnts = getGeoIntDiagBox(geo)
    for pnt in pnts:
        print(pnt)
        folium.Circle(pnt, color='darkblue', radius=1500, fill=False, weight=1, opacity=1).add_to(m)

    pnts = getGeoIntQuadBox(geo)
    for pnt in pnts:
        print(pnt)
        folium.Circle(pnt, color='darkblue', radius=1500, fill=False, weight=1, opacity=1).add_to(m)

    
    pnts = getGeoPrimBBox(geo)
    for pnt in pnts:
        folium.Circle(pnt, color='darkblue', radius=500, fill=False, weight=1, opacity=1).add_to(m)

    pnts = getGeoDiagBBox(geo)
    for pnt in pnts:
        folium.Circle(pnt, color='darkgreen', radius=500, fill=False, weight=1, opacity=1).add_to(m)

    pnts = getGeoQuadBBox(geo)
    for pnt in pnts:
        folium.Circle(pnt, color='darkpurple', radius=500, fill=False, weight=1, opacity=1).add_to(m)

    pnts = getGeoOctoBBox(geo)
    for pnt in pnts:
        folium.Circle(pnt, color='green', radius=500, fill=False, weight=1, opacity=1).add_to(m)

    pnts = getGeoDeciBBox(geo)
    for pnt in pnts:
        folium.Circle(pnt, color='purple', radius=500, fill=False, weight=1, opacity=1).add_to(m)

if False:
    geoPnts = []
    for geo in geos:
        pnts = getGeoDiagBBox(geo)
        geoPnts.append(pnts)
    #print(pnt.coords)
    folium.PolyLine(geoPnts, color='blue', weight=2, opacity=1).add_to(m)

    geoPnts = []
    for geo in geos:
        pnts = getGeoQuadBBox(geo)
        geoPnts.append(pnts)
    #print(pnt.coords)
    folium.PolyLine(geoPnts, color='green', weight=2, opacity=1).add_to(m)

    geoPnts = []
    for geo in geos:
        pnts = getGeoOctoBBox(geo)
        geoPnts.append(pnts)
    #print(pnt.coords)
    folium.PolyLine(geoPnts, color='purple', weight=2, opacity=1).add_to(m)

    geoPnts = []
    for geo in geos:
        pnts = getGeoDeciBBox(geo)
        geoPnts.append(pnts)
    #print(pnt.coords)
    folium.PolyLine(geoPnts, color='orange', weight=2, opacity=1).add_to(m)


m

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ImportError: cannot import name 'getGeoIntPrimBox'

In [56]:
import string
import random
def id_generator(num=1000000, size=7, chars=string.ascii_lowercase):
    retvals = []
    for i in range(num):
        retvals.append(''.join(random.choice(chars) for _ in range(size)))
    return retvals

In [57]:
vals = id_generator()

In [58]:
a=[x[:6] for x in vals]
from collections import Counter
cnt = Counter(a)

In [50]:
from itertools import groupby
x = {key: len(list(group)) for key, group in groupby(a)}
#groups = groupby(a)
#[len(list(group)) for key, group in groups]
#for key, group in groups:
#    print(key,list(group))

In [60]:
cnt.most_common()

[('epwetj', 2),
 ('pwratn', 2),
 ('zpfmhp', 2),
 ('diowfs', 2),
 ('funbvg', 2),
 ('uukdzv', 2),
 ('oimbmn', 2),
 ('uewssr', 2),
 ('bpsscy', 2),
 ('ylmamh', 2),
 ('cpeqrd', 2),
 ('xzsked', 2),
 ('fdrnzp', 2),
 ('mpnvsg', 2),
 ('kuwnbc', 2),
 ('kpoujt', 2),
 ('lfbaqn', 2),
 ('yojqbj', 2),
 ('ylfmns', 2),
 ('xzecnu', 2),
 ('pilfuw', 2),
 ('timusa', 2),
 ('wdfdlu', 2),
 ('ssmwpd', 2),
 ('rdrefa', 2),
 ('rrfxql', 2),
 ('eibptz', 2),
 ('oywqcb', 2),
 ('zeizlu', 2),
 ('jvrphj', 2),
 ('efjmxx', 2),
 ('ebkegx', 2),
 ('hbdrzc', 2),
 ('sjkhps', 2),
 ('wnfrye', 2),
 ('bbtird', 2),
 ('aglwrm', 2),
 ('steqib', 2),
 ('qinfwb', 2),
 ('zovgjt', 2),
 ('wsizam', 2),
 ('tmbwys', 2),
 ('pjtccg', 2),
 ('pipbio', 2),
 ('ogvuug', 2),
 ('qpzdxz', 2),
 ('peswls', 2),
 ('shrwet', 2),
 ('tzgeeb', 2),
 ('svekln', 2),
 ('opnfig', 2),
 ('wxybyb', 2),
 ('eodpba', 2),
 ('pwfgvt', 2),
 ('gevpet', 2),
 ('tvqxku', 2),
 ('bqxlkv', 2),
 ('nylbgv', 2),
 ('vvorsa', 2),
 ('eqfjlc', 2),
 ('vaffme', 2),
 ('xbxuya', 2),
 ('rhszi

In [5]:
from datetime import datetime as dt
import datetime
try:
    from haversine import haversine
except:
    raise ValueError("Must have haversine installed!")
from random import random
from numpy import ceil
try:
    from shapely.geometry import Point
    from shapely.geometry.polygon import Polygon
except:
    raise ValueError("Must have shapely installed!")
from pygeohash import bbox, decode_exactly, encode, getGeoChars
import json
              

def clock(comment=None,showTime=True):
    val = dt.now()
    if comment is None:
        comment = "Begin"
    if showTime is True:
        print("Current Time is {0} for {1}".format(printDateTime(val, "%a %b %d, %Y %H:%M:%S"), comment))
    return val, comment


def elapsed(start,comment=None,showTime=True):
    if isinstance(start, tuple):
        comment = start[1]
        start   = start[0]        
    if comment is not None:
        if not comment.startswith("Done with"):
            comment = "Done with {0}".format(comment)
        
    if comment is None:
        comment = "End"
    end, _ = clock(comment=comment, showTime=showTime)
    dtime = end-start
    dtime = dtime.seconds
    if dtime>60:
        print("Process [{0}] took {1} minutes.".format(comment, round(dtime/60.0,1)))
    else:
        print("Process [{0}] took {1} seconds.".format(comment, round(dtime/1.0,1)))

def printDateTime(dtval,dtformat="%Y-%m-%d"):
    if isinstance(dtval,str):
        dtobj = getDateTime(dtval)
    else:
        dtobj = dtval
        
    if isinstance(dtobj, datetime.datetime):
        dtstr = dt.strftime(dtobj, dtformat)
    elif isinstance(dtobj, datetime.date):
        dtstr = dt.strftime(dtobj, dtformat)
    else:
        print("Can not print object {0}".format(dtval))
        dtstr = dtval
              
    return dtstr

def getBBox(shape):
    bbox = [float('%.6f' % coord) for coord in shape.bbox]
    return bbox


def getLatRange(bbox):
    if isinstance(bbox[1], (int, float)):
        oLat  = min([bbox[1], bbox[3]])
        dLat  = abs(bbox[1] - bbox[3])
    elif isinstance(bbox[1], tuple):
        oLat  = min([bbox[1][0], bbox[3][0]])
        dLat  = abs(bbox[1][0] - bbox[3][0])
        
    return round(dLat,4),round(oLat,4)


def getLngRange(bbox):
    if isinstance(bbox[0], (int, float)):
        oLng  = min([bbox[0], bbox[2]])
        dLng  = abs(bbox[0] - bbox[2])
    elif isinstance(bbox[0], tuple):
        oLng  = min([bbox[0][1], bbox[2][1]])
        dLng  = abs(bbox[0][1] - bbox[2][1])

    return round(dLng,4),round(oLng,4)


def addShapeGeos(shape, prec, bbox = None, maxmiss = None, debug = False):    
    verydebug=False
    
    ## Create Polygon
    poly      = Polygon(shape.points)
    polyarea  = poly.area
        
    geos = {}
    geoPnts = []
    if isinstance(bbox, str):
        #######################################################################################################################################
        ## Geo boundary approach
        #######################################################################################################################################
        
        maingeo   = bbox
        geoarea   = getGeoArea(len(maingeo))
        ngeoguess = int(ceil(polyarea / geoarea))
        nrounds   = 32
        nmiss     = 0
        testGeos = ["{0}{1}".format(maingeo, x) for x in getGeoChars()]
        for testGeo in testGeos:
            geos[testGeo] = isGeoInShape(geo=testGeo, poly=poly)
    else:
        #######################################################################################################################################
        ## Traditional Monte Carlo approach
        #######################################################################################################################################
        if bbox is None:
            bbox = getBBox(shape)
        dLat,oLat = getLatRange(bbox)
        dLng,oLng = getLngRange(bbox)
        area      = dLat * dLng
        geoarea   = getGeoArea(prec)

        ## Guess
        ngeoguess = int(ceil(area / geoarea))
        minMR = 2500
        if maxmiss is None:
            maxmiss = min([max([ngeoguess,100]), minMR])
        if verydebug:
            print("Area of BBox is {0}, Geo area is {1}, and guessing there are {2} geos in this bbox".format(round(area,4), round(geoarea, 4), ngeoguess))
            print("Area of Polygon is {0}".format(round(polyarea,4)))
            print("Latitude Range:  {0} and offset {1}".format(dLat, oLat))
            print("Longitude Range: {0} and offset {1}".format(dLng, oLng))

        nmiss=0
        nrounds   = 0
        maxR = 100000
        maxrounds = min([max([20*ngeoguess, 100]), maxR])
        maxrounds = ceil(int(1000/prec))
        maxmiss   = ceil(int(1000/prec))

        maxmiss   = 100
        maxrounds = ceil(int(60000/prec))
        maxmiss   = maxrounds/20
        while nrounds < maxrounds:
            nrounds += 1

            ## Generate Lat, Lng
            lat = dLat*random() + oLat
            lng = dLng*random() + oLng

            ## See if we need to add it
            pnt = Point(lng,lat)
            geo = encode(latitude=lat, longitude=lng, precision=prec)
            if geos.get(geo) is None:
                geos[geo] = {}
            #if verydebug:
            #    print("Round {0}: ({1},{2}) --> {3} with result {4}".format(nrounds, round(lat,4), round(lng,4), geo, geos.get(geo)))
            if poly.contains(pnt):
                if geos[geo].get('I') is None:
                    geos[geo]["I"] = 1
                    if verydebug:
                        print("\tAdding IN for {0} and Reseting".format(geo))
                    nmiss     = 0
                else:
                    nmiss += 1
            else:
                if geos[geo].get("O") is None:
                    geos[geo]["O"] = 1
                    if verydebug:
                        print("\tAdding OUT for {0} and Reseting".format(geo))
                    nmiss  = 0
                else:
                    nmiss += 1

            if verydebug:
                print("After round {0}/{1}, there are {2} geos and {3}/{4} misses".format(nrounds, maxrounds, len(geos), nmiss, maxmiss))

            ## Check for a max number of misses
            if nmiss > maxmiss:
                if verydebug:
                    print("Breaking after {0}/{1} rounds collecting {2} geos".format(nrounds, maxrounds, len(geos)))
                break
            
            
    #######################################################################################################################################
    ## Delete all geos where we didn't find an inside point
    dels=[]
    for geo,geodata in geos.items():
        if geodata.get("I") is None:
            dels.append(geo)
    for geo in dels:
        del geos[geo]
    
    if debug:
        print("    Found {0} geos with guess {1} after {2} rounds and {3} misses".format(len(geos), ngeoguess, nrounds, nmiss))
    return geos

def isMaxPrec(curprec, maxPrec, debug=False):
    if curprec >= maxPrec:
        if debug:
            print("\tPrec[{0}] >= maxPrec[{1}]".format(curprec, maxPrec))      
        return True
    return False

def precStatus(curprec, geos, debug=False):
    if debug:
        print("Total {0} Geos: {1}".format(curprec, len(geos)))  
        

def isInside(geodata):
    if geodata.get("I") is None:
        return False
    return True

def isOutside(geodata):
    if geodata.get("O") is None:
        return False
    return True


    

############################################################################################################################################
#
# Current Method of Starting with Large -> Small Geos
#
############################################################################################################################################
def getShapeGeos(shape, maxPrec=8, debug=False, verydebug=False):
    
    startPrec = guessStartPrec(shape, debug=debug)
    if debug:
        print("Start Prec -> {0}".format(startPrec))
    
    adds = {}
    prec = startPrec
    dels = {0: set(), 1: set(), 2: set(), 3: set(), 4: set()}
    verydebug=verydebug
    
    
    ## Add one point we know is in the shape
    geos = getInitGeo(shape, prec, debug)
    

    ## Report what we found and check if we should stop
    precStatus(prec+0, geos, debug)
    if isMaxPrec(prec+1, maxPrec, debug):
        return list(geos.keys())
    
    
    ## Loop over found geos
    for geo0 in list(geos.keys()):
        geoval0 = geos[geo0]
        
        ## check inside
        if not isInside(geoval0):
            dels[0].add(geo0) # remove geo because nothing was inside
            continue
            
        ## check outside (must already be inside)
        if not isOutside(geoval0):
            continue # no outside points so this one is done
            
        ## this should always pass since we checked the previous two
        if isOutside(geoval0):
            dels[0].add(geo0) # remove point because we are going to loop over values inside it with a higher precision
            bbox       = getGeoBBox(geo0)
            ## EDIT!!!
            adds[geo0] = addShapeGeos(shape, prec=prec+1, bbox = geo0, maxmiss = None, debug = debug) # keep everything within the bounding box
            geos.update(adds[geo0])
    

            ## Report what we found and check if we should stop
            precStatus(prec+1, geos, debug)
            if isMaxPrec(prec+2, maxPrec, debug):
                continue
            

            ## Loop over found geos
            for geo1 in list(adds[geo0].keys()):
                geoval1 = geos[geo1]
        
                ## check inside
                if not isInside(geoval1):
                    dels[1].add(geo1) # remove geo because nothing was inside
                    continue

                ## check outside (must already be inside)
                if not isOutside(geoval1):
                    continue # no outside points so this one is done
            
                ## this should always pass since we checked the previous two
                if isOutside(geoval1):
                    dels[1].add(geo1) # remove point because we are going to loop over values inside it with a higher precision
                    bbox       = getGeoBBox(geo1)
                    ## EDIT!!!
                    adds[geo1] = addShapeGeos(shape, prec=prec+2, bbox = geo1, maxmiss = None, debug = debug) # keep everything within the bounding box
                    geos.update(adds[geo1])
    

                    ## Report what we found and check if we should stop
                    precStatus(prec+2, geos, debug)
                    if isMaxPrec(prec+3, maxPrec, debug):
                        continue


                    ## Loop over found geos
                    for geo2 in list(adds[geo1].keys()):
                        geoval2 = geos[geo2]

                        ## check inside
                        if not isInside(geoval2):
                            dels[2].add(geo2) # remove geo because nothing was inside
                            continue

                        ## check outside (must already be inside)
                        if not isOutside(geoval2):
                            continue # no outside points so this one is done

                        ## this should always pass since we checked the previous two
                        if isOutside(geoval2):
                            dels[2].add(geo2) # remove point because we are going to loop over values inside it with a higher precision
                            bbox       = getGeoBBox(geo2)
                            ## EDIT!!!
                            adds[geo2] = addShapeGeos(shape, prec=prec+3, bbox = geo2, maxmiss = None, debug = debug) # keep everything within the bounding box
                            geos.update(adds[geo2])
    

                            ## Report what we found and check if we should stop
                            precStatus(prec+3, geos, debug)
                            if isMaxPrec(prec+4, maxPrec, debug):
                                continue


                            ## Loop over found geos
                            for geo3 in list(adds[geo2].keys()):
                                geoval3 = geos[geo3]

                                ## check inside
                                if not isInside(geoval3):
                                    dels[3].add(geo3) # remove geo because nothing was inside
                                    continue

                                ## check outside (must already be inside)
                                if not isOutside(geoval3):
                                    continue # no outside points so this one is done

                                ## this should always pass since we checked the previous two
                                if isOutside(geoval3):
                                    dels[3].add(geo3) # remove point because we are going to loop over values inside it with a higher precision
                                    bbox       = getGeoBBox(geo3)
                                    ## EDIT!!!
                                    adds[geo3] = addShapeGeos(shape, prec=prec+4, bbox = geo3, maxmiss = None, debug = debug) # keep everything within the bounding box
                                    geos.update(adds[geo3])
                            
                            

                            
    if debug:
        print("Total Keys Before Del {0} : {1}".format(prec+0, len(geos)))
    for geo in dels[0]:
        if len(geos) > 1:
            if debug: print("\tDel {0}".format(geo))
            del geos[geo]
    if debug:
        print("Total Keys Before Del {0} : {1}".format(prec+1, len(geos)))
    for geo in dels[1]:
        if len(geos) > 1:
            if debug: print("\tDel {0}".format(geo))
            del geos[geo]
    if debug:
        print("Total Keys Before Del {0} : {1}".format(prec+2, len(geos)))
    for geo in dels[2]:
        if len(geos) > 1:
            if debug: print("\tDel {0}".format(geo))
            del geos[geo]
    if debug:
        print("Total Keys Before Del {0} : {1}".format(prec+3, len(geos)))
    for geo in dels[3]:
        if len(geos) > 1:
            if debug: print("\tDel {0}".format(geo))
            del geos[geo]
    if debug:
        print("Total Keys: {0}".format(len(geos)))
    

    dels = []
    for r in range(startPrec,maxPrec+1):
        rgeos = [x for x,v in geos.items() if len(x) == r]
        for rgeo in rgeos:
            dels += [x for x,v in geos.items() if len(x) > len(rgeo) and x.startswith(rgeo)]

    for geo in dels:
        if debug: print("\tDel {0}".format(geo))
        if len(geos) > 1:
            try:
                del geos[geo]            
            except:
                continue
            
    if verydebug:
        print("Found {0} extra guys to delete".format(len(dels)))
    
    if debug:
        print("")
        for r in range(4,9):
            ins  = ([x for x,v in geos.items() if len(x) == r and v.get('O') is None])
            outs = ([x for x,v in geos.items() if len(x) == r and v.get('O') is not None])

            print("========== {0} ==========".format(r))
            print("\tIN:  {0}".format(json.dumps(sorted(ins))))
            print("\tOUT: {0}".format(json.dumps(sorted(outs))))
            if False:
                for out in sorted(outs):
                    print("\t\t== {0} ==".format(out))
                    subsIN  = [x for x,v in geos.items() if len(x) == r+1 and x.startswith(out) and v.get('O') is None]
                    subsOUT = [x for x,v in geos.items() if len(x) == r+1 and x.startswith(out) and v.get('O') is not None]
                    print("\t\tSUB IN:  {0}".format(json.dumps(sorted(subsIN))))
                    print("\t\tSUB OUT: {0}".format(json.dumps(sorted(subsOUT))))

            print("")
            
    return list(geos.keys())

_ = clock("Last Run")

Current Time is Wed Oct 10, 2018 14:25:37 for Last Run
